In [3]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy
from context import *

In [29]:
for k in range(10000):
    tmp = random_clifford_state(2)
    if utils.acq_mat(tmp.gs[:2]).sum()!=0:
        print('ERROR @ ', k)
        print(tmp)
        break

ERROR @  11
StabilizerState(
   +XZ
   +IX)


In [111]:
for k in range(10000):
    tmp = random_clifford_map(3)
    if utils.acq_mat(tmp.gs[1::2]).sum()!=0:
        print('ERROR @ ', k)
        print(tmp)
        break

ERROR @  0
CliffordMap(
  X0-> -ZI
  Z0-> -YZ
  X1-> +IZ
  Z1-> -IY)


In [109]:
for k in range(10000):
    gs = utils.random_clifford(2)
    acq = utils.acq_mat(gs)
    if (acq - numpy.kron(numpy.eye(2,dtype=int),numpy.array([[0,1],[1,0]]))).sum() != 0:
        print('ERROR @ ', k)
        print(gs)
        print(acq)
        break

In [104]:
def random_pair(N):
    '''Sample an anticommuting pair of random stabilizer and destabilizer.
    Parameters:
    N: int - number of qubits.
    Returns:
    g1: int (2*N) - binary representation of stabilizer.
    g2: int (2*N) - binary representation of destabilizer.
    '''
    g1 = numpy.random.randint(0,2,2*N)
    g2 = numpy.random.randint(0,2,2*N)
    while (g1 == 0).all(): # resample g1 if it is all zero
        g1 = numpy.random.randint(0,2,2*N)
    if utils.acq(g1, g2) == 0: # if g1, g2 commute
        i = utils.front(g1) # locate the first nontrivial g1 site
        # flip commutativity by chaning g2
        g2[2*i] = (g2[2*i] + g1[2*i+1])%2
        g2[2*i+1] = (g2[2*i+1] + g1[2*i] + g1[2*i+1])%2
    return g1, g2


# def random_clifford_(gs):
#     '''Fill random anticommuting Pauli strings in an array.
#         (as recursive constructor called by random_clifford) 
#     Parameters:
#     gs: int (2*n, 2*n) - array to fill in with Pauli strings.'''
#     n = gs.shape[-1]//2
#     print('-------\nenter with n = ', n)
#     g1, g2 = random_pair(n)
#     print('g1 = ', g1)
#     print('g2 = ', g2)
#     if n == 1:
#         gs[0] = g1
#         gs[1] = g2
#     else:
#         gens, g1, g2 = pauli_diagonalize2(g1, g2)
#         print('diagonalize')
#         print(gens)
#         print('g1 = ', g1)
#         print('g2 = ', g2)
#         gs[0] = g1
#         gs[1] = g2
#         random_clifford_(gs[2:,2:])
#         print('expand:')
#         print(gs)
#         for g in reversed(gens):
#             gs = utils.clifford_rotate_signless(g, gs)
#     print('output:')
#     print(gs)
#     return gs

def random_clifford_(gs):
    '''Fill random anticommuting Pauli strings in an array.
        (as recursive constructor called by random_clifford) 
    Parameters:
    gs: int (2*n, 2*n) - array to fill in with Pauli strings.'''
    n = gs.shape[-1]//2
    g1, g2 = random_pair(n)
    if n == 1:
        gs[0] = g1
        gs[1] = g2
    else:
        gens, g1, g2 = pauli_diagonalize2(g1, g2)
        gs[0] = g1
        gs[1] = g2
        random_clifford_(gs[2:,2:])
        for g in reversed(gens):
            gs = utils.clifford_rotate_signless(g, gs)
    return gs

In [102]:
numpy.random.seed(33)
gs = random_clifford_(numpy.zeros((2*2,2*2), dtype=numpy.int_))
acqmat = utils.acq_mat(gs)
print('gs:\n',gs)
print('acq:\n',acqmat)

-------
enter with n =  2
g1 =  [0 1 0 0]
g2 =  [1 0 1 1]
diagonalize
[array([0, 1, 1, 1])]
g1 =  [0 1 0 0]
g2 =  [1 1 0 0]
-------
enter with n =  1
g1 =  [0 1]
g2 =  [1 0]
output:
[[0 1]
 [1 0]]
expand:
[[0 1 0 0]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]
output:
[[0 1 0 0]
 [1 0 1 1]
 [0 1 1 0]
 [0 1 0 1]]
gs:
 [[0 1 0 0]
 [1 0 1 1]
 [0 1 1 0]
 [0 1 0 1]]
acq:
 [[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]


In [107]:
for k in range(10000):
    numpy.random.seed(k)
    gs = utils.random_clifford_(numpy.zeros((2*2,2*2), dtype=numpy.int_))
    acqmat = utils.acq_mat(gs)
    if (acqmat - numpy.kron(numpy.eye(2,dtype=int),numpy.array([[0,1],[1,0]]))).sum() != 0:
        print('ERROR @ ', k)
        print(gs)
        print(acqmat)
        break

AttributeError: module 'vaeqst.utils' has no attribute 'random_clifford_'

In [93]:
def pauli_diagonalize2(g1, g2, i0 = 0):
    '''Find a series of Clifford roations to diagonalize a pair of anticommuting
    Pauli strings to qubit i0 as Z and X (or Y).
    Parameters:
    g1: int (2*N) - binary representation of stabilizer.
    g2: int (2*N) - binary representation of destabilizer.
    i0: int - target qubit
    Returns:
    gs: int (L, 2*N) - binary representations of Clifford generators.
    g1: int (2*N) - binary representation of transformed stabilizer.
    g2: int (2*N) - binary representation of transformed destabilizer.'''
    assert g1.shape == g2.shape
    (N2,) = g1.shape
    N = N2//2
    gs = [] # prepare to collect Clifford generators
    # bring g1 to Z0
    if utils.pauli_is_onsite(g1, i0): # if g1 is on-site: g1 = X0,Y0,Z0
        if g1[2*i0] == 1: # if g1 = X0,Y0
            g = g1.copy() 
            g[2*i0+1] = (g[2*i0+1] + 1)%2
            # now g = Y0,X0
            gs.append(g)
            g1 = (g1 + g)%2
            g2 = (g2 + utils.acq(g, g2) * g)%2
            # now g1 has been transformed to Z0
    else: # if g1 is not on-site
        if g1[2*i0] == 0: # if g1 commute with Z0
            g = g1.copy()
            if g1[2*i0+1] == 0: # g1 is trivial at site 0
                i = utils.front(g) # find the first non-trivial qubit as pivot
                # XYZ cyclic on the pivot qubit
                g[2*i] = (g[2*i] + g[2*i+1])%2
                g[2*i+1] = (g[2*i+1] + g[2*i])%2
                # now g anticommute with g1
                g[2*i0] = 1 # such that g also anticommute with Z0     
            else: # g1 has Z at site 0
                g[2*i0] = 1 # such that g anticommute with both g1, Z0
            gs.append(g)
            g1 = (g1 + g)%2
            g2 = (g2 + utils.acq(g, g2) * g)%2
        # now g1 anticommute with Z0                
        g = g1.copy()
        g[2*i0+1] = (g[2*i0+1] + 1)%2 # g = g1 (*) Z0
        gs.append(g)
        g1 = (g1 + g)%2
        g2 = (g2 + utils.acq(g, g2) * g)%2
        # now g1 has been transformed to Z0
    # bring g2 to X0,Y0
    if not utils.pauli_is_onsite(g2, i0):
        g = g2.copy()
        g[2*i0] = 0
        g[2*i0+1] = 1
        gs.append(g)
        g2 = (g2 + g)%2
        # now g2 has been transformed to X0 or Y0
    return gs, g1, g2

In [94]:
pauli_diagonalize2(numpy.array([0,1,0,0]), numpy.array([1,0,1,1]))

([array([0, 1, 1, 1])], array([0, 1, 0, 0]), array([1, 1, 0, 0]))

In [122]:
p = - 2*pauli('XI')+pauli('IX')+5*pauli('YY')+3*pauli('ZZ')+2*pauli('XX')
p

1 IX +3 ZZ -2 XI +2 XX +5 YY

In [124]:
(p @ p).reduce()

43 II -8 IX -20 ZZ +4 XI -34 XX -12 YY

In [116]:
ham = pauli_zero(4)
for i in range(4):
    ham += numpy.random.rand() * pauli({i:'Z',(i+1)%4:'Z'}, 4)
    ham += numpy.random.rand() * pauli({i:'X'}, 4)
ham

0.77 IIIX +0.40 IIZZ +0.82 IIXI +0.65 IZZI +0.24 IXII +0.87 ZIIZ +0.69 ZZII +0.02 XIII

In [120]:
heff, circ = circuit.SBRG(ham)
heff, circ

(1.18 IZII -0.32 IZZI -1.21 ZIII +0.73 ZIZI +0.00 ZZIZ, QuantumCircuit(
   |[3]|
   |[2,3]|
   |[0][1,2]|
   |[0,3][1,2]|))

In [121]:
circ.forward(ham.copy())

0.77 YIYZ -0.40 IXZI +0.82 IZII +0.65 IXII -0.24 IZXX -0.87 ZIII +0.69 ZIZI -0.02 XIIX

In [105]:
p = pauli('YY')
circ = diagonalize(p)
circ, p

(QuantumCircuit(
   |[0,1]|),  +YY)

In [25]:
circuit.diagonalize(pauli('IIZII'),0)

QuantumCircuit(
  |[0,2]|
  |[0,2]|)

In [48]:
circ.forward(p)

-1 XZ +3 ZZ -2 XI -2 IZ +5 ZI

In [79]:
diagonalize(pauli('ZZIIX'),2,after=True)

QuantumCircuit(
  |[2,4]|
  |[2,4]|)

In [78]:
diagonalize(pauli('ZZIIX'),2,after=True).forward(paulis('ZZIIX'))

 +ZZZII

In [77]:
diagonalize(pauli('ZZIIX'),2,after=False).forward(paulis('ZZIIX'))

 +IIZII

In [28]:
circ.forward(paulis('ZIZII'))

 -ZIIII

In [29]:
circ.last_layer.forward_map.gs

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [37]:
p.gs[numpy.abs(p.cs) > 1.e-10]

array([[1, 0]])

In [75]:
numpy.array([4.], dtype=numpy.complex_)

array([4.+0.j])

In [70]:
numpy.expand_dims(pauli('XXX').g,0)

array([[1, 0, 1, 0, 1, 0]])

In [50]:
stabilizer_state('XXXXXX','-ZZIIII')

StabilizerState(
   +ZZIIII
   +XXXXXX)

In [4]:
vaeqst.ghz_state(6).expect(vaeqst.paulis('XXXXXX','-ZZIIII'))

array([ 1, -1])

In [7]:
a = vaeqst.utils.random_clifford(500)

In [8]:
vaeqst.utils.z2inv(a)

array([[1, 1, 1, ..., 1, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 1, 0],
       ...,
       [1, 1, 0, ..., 0, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 0, ..., 0, 1, 0]])

In [9]:
a = vaeqst.utils.random_clifford(500)
diff = vaeqst.utils.z2inv(a) @ a % 2 - numpy.eye(a.shape[0], dtype=a.dtype)
numpy.all(diff == 0)

True

In [15]:
rho = vaeqst.random_clifford_state(5)
rho

StabilizerState(
   -ZXIYX
   +IIYII
   +YZIZZ
   +YYYIX
   -YZIXY)

In [16]:
circ = vaeqst.diagonalize(rho)
circ

QuantumCircuit(
  |[0,1,2,3,4]|)

In [17]:
circ.forward(rho.copy())

StabilizerState(
   +ZIIII
   +IZIII
   +IIZII
   +IIIZI
   +IIIIZ)

In [18]:
circ.backward(vaeqst.zero_state(5))

StabilizerState(
   -ZXIYX
   +IIYII
   +YZIZZ
   +YYYIX
   -YZIXY)

In [19]:
circ.first_layer.gates[0].forward_map

CliffordMap(
  X0-> -YZYXX
  Z0-> -ZIYXX
  X1-> -IIXXY
  Z1-> +XZZXI
  X2-> -ZXZXZ
  Z2-> +ZYZXZ
  X3-> -XZXZZ
  Z3-> -XIIIY
  X4-> +IZYZY
  Z4-> +XIIXY)

In [70]:
circ.forward(vaeqst.paulis('-ZZIII','IIXII'))

 +YIZZX
 +XZZZZ

In [18]:
numpy.arange(5)

array([0, 1, 2, 3, 4])

In [12]:
ghz_state(5).expect(paulis('-ZZIII','IIXII'))

array([-1.,  0.])

In [13]:
stabilizer.ghz_state(5).expect(stabilizer.zero_state(5))

[3 3 3 3 3]


0.03125

In [10]:
stabilizer.ghz_state(10).entropy(utils.mask([2,3,4],10))

1

In [9]:
stabilizer.maximally_mixed_state(10).entropy(utils.mask([2,3,6,7],10))

4

In [12]:
ghz_state(4).stabilizer_group()

 +IIII
 +XXXX
 +IIZZ
 -XXYY
 +IZZI
 -XYYX
 +IZIZ
 -XYXY
 +ZZII
 -YYXX
 +ZZZZ
 +YYYY
 +ZIZI
 -YXYX
 +ZIIZ
 -YXXY

In [15]:
utils.z2rank(numpy.array([[0,1,0,0],[0,0,0,1],[1,1,1,1]]))

3

In [4]:
qdvc = QuantumDevice(ghz_state(6), brickwall_rcc(6,2))
qdvc

QuantumDevice(
  StabilizerState(
     +ZZIIII
     +IZZIII
     +IIZZII
     +IIIZZI
     +IIIIZZ
     +XXXXXX),
  QuantumCircuit(
    |[1,2][3,4][5,0]|
    |[0,1][2,3][4,5]|))

In [53]:
list(qdvc.measure(2))

[StabilizerState(
    -IYXXYY
    -IZZXXX
    -XIIXII
    -ZYZYYY
    -IIXXYI
    -IIZXZY), StabilizerState(
    -IYXXYY
    -IZZXXX
    -XIIXII
    -ZYZYYY
    -IIXXYI
    -IIZXZY)]

In [97]:
a = numpy.array([[1,1,1,1]])
a[:,numpy.array([True,False,True,False])] = numpy.array([[2,2]])
a

array([[2, 1, 2, 1]])

In [73]:
numpy.logical_and(numpy.array([True,True,False,False]),numpy.array([True,True,False,False])).dtype

dtype('bool')

In [6]:
numpy.repeat(utils.mask((2,3),5),2)

array([False, False, False, False,  True,  True,  True,  True, False,
       False])

In [104]:
gen = pauli('XI')
obs = paulis('YX','ZY')
utils.clifford_rotate(gen.g,gen.p,obs.gs,obs.ps)
obs

 +ZX
 -YY

In [105]:
gen = pauli('X')
obs = paulis('YX','ZY')
msk = numpy.repeat(utils.mask(0,2),2)
#utils.clifford_rotate(gen.g,gen.p,obs.gs[:,msk],obs.ps)
obs.gs[:,msk],obs.ps = utils.clifford_rotate(gen.g,gen.p,obs.gs[:,msk],obs.ps)
obs

 +ZX
 -YY

In [108]:
gen = pauli('X')
obs = paulis('YX','ZY')
obs.rotate_by(gen,utils.mask((0,),2))

 +ZX
 -YY

In [132]:
m0 = identity_map(4)
m1 = random_clifford_map(2)
m0, m1

(CliffordMap(
   X0-> +XIII
   Z0-> +ZIII
   X1-> +IXII
   Z1-> +IZII
   X2-> +IIXI
   Z2-> +IIZI
   X3-> +IIIX
   Z3-> +IIIZ), CliffordMap(
   X0-> +XY
   Z0-> -ZI
   X1-> +ZZ
   Z1-> +IY))

In [134]:
m0.embed(m1, utils.mask((2,3),4))

CliffordMap(
  X0-> +XYII
  Z0-> -ZIII
  X1-> +ZZII
  Z1-> +IYII
  X2-> +IIXY
  Z2-> -IIZI
  X3-> +IIZZ
  Z3-> +IIIY)

In [119]:
msk = numpy.repeat(utils.mask((0,1),4),2)
msk

array([ True,  True,  True,  True, False, False, False, False])

In [130]:
m0.gs[numpy.ix_(msk,msk)] = m1.gs

In [131]:
m0

CliffordMap(
  X0-> +YIII
  Z0-> +ZZII
  X1-> +IZII
  Z1-> +YXII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

In [18]:
utils.stabilizer_across(paulis('IIIX','IYZX').gs, utils.mask((1,2),4))

array([[1, 1, 0, 1]])

In [48]:
%load_ext snakeviz

In [58]:
%%snakeviz
N = 50
nsample = 100
qdvc = vaeqst.QuantumDevice(vaeqst.ghz_state(N), vaeqst.global_rcc(N))
for snapshot in qdvc.measure(nsample):
    pass

 
*** Profile stats marshalled to file '/var/folders/1m/3nz1kxmj2mgb2s2gwq2ndxqh0000gn/T/tmp3au98zkk'. 
Embedding SnakeViz in this document...


In [3]:
stabilizer.stabilizer_state('XXI','XIZ')

StabilizerState(
 +XIZ
 +XXI)

In [4]:
stabilizer.stabilizer_state('XXI','XIZ').stabilizers

 +XIZ
 +XXI

In [5]:
stabilizer.stabilizer_state('XXI','XIZ')[:]

 +IIZ
 +XIZ
 +XXI
 +ZZX
 +ZZI
 +IZI

In [6]:
st1 = stabilizer.ghz_state(3)
st2 = stabilizer.stabilizer_state('XXI','XXZ')
st1.measure(st2)

(array([1, 1]), -2.0)

In [34]:
st1 = stabilizer.ghz_state(3)
st2 = pauli.paulis('XXI','XIZ')
utils.stabilizer_measure(st1.gs, st1.ps, st2.gs, st2.ps, st1.r)

(array([[1, 0, 0, 0, 0, 1],
        [1, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 1]]), array([2, 0, 2, 0, 0, 0]), 0, array([0, 1]), -2)

In [128]:
st1

StabilizerState(
 +ZZI
 -XXZ
 +XXI)

In [134]:
st1.stabilizer_group()

 +III
 +XXI
 -XXZ
 -IIZ
 +ZZI
 -YYI
 +YYZ
 -ZZZ

In [117]:
st1.entropy(numpy.array([0,1,0]))

1.0

In [103]:
circ = circuit.brickwall_rcc(6, 2)
circ

|[1,2][3,4][5,0]|
|[0,1][2,3][4,5]|

In [253]:
utils.acq_mat(utils.random_pauli_tableau(4))

array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0]])

In [13]:
circuit.identity_circuit(5)

QuantumCircuit(
  ||)

In [16]:
circ = circuit.diagonalize(pauli('IIXI'),0)
circ

QuantumCircuit(
  |[0,2]|
  |[0,2]|)

In [18]:
os = paulis('+IIXI','+YIXX','-IZYZ','+XIIY')
os

 +IIXI
 +YIXX
 -IZYZ
 +XIIY

In [239]:
circ.forward(os)

 +ZIII
 +IIYX
 +YZXZ
 -ZIZY

In [11]:
clifford.random_clifford_map(2)

CliffordMap(
  X0-> -XX
  Z0-> +IY
  X1-> -ZY
  Z1-> -XI)

In [53]:
str(int(numpy.ceil(numpy.log10(11))))

'2'

In [19]:
gen = pauli('ZIZI')
gen

 +ZIZI

In [236]:
os.copy().rotate_by(gen)

 +ZIYI
 +YIXX
 +ZZXZ
 +YIZY

In [222]:
gate = circuit.CliffordRotation(gen)
gate.clifford_generator, gate.qubits

( +ZZ, (0, 2))

In [237]:
os.copy().rotate_by(gate.clifford_generator, gate.qubits)

 +ZIYI
 +YIXX
 +ZZXZ
 +YIZY

In [224]:
gate.compile()
gate.clifford_forward

 +YZ
 +ZI
 +ZY
 +IZ

In [238]:
os.copy().transform_by(gate.clifford_forward, gate.qubits)

 +ZIYI
 +YIXX
 +ZZXZ
 +YIZY

In [101]:
tuple(range(2))

(0, 1)

In [136]:
%load_ext snakeviz

In [47]:
circ.last_layer.gates[0].clifford_generator

 +YZ

In [49]:
circ.last_layer.gates[0].qubits

(0, 2)

In [56]:
cm.inverse().compose(cm)

 +XIII
 +ZIII
 +IXII
 +IZII
 +IIXI
 +IIZI
 +IIIX
 +IIIZ

In [57]:
cm.compose(cm.inverse())

 +XIII
 +ZIII
 +IXII
 +IZII
 +IIXI
 +IIZI
 +IIIX
 +IIIZ

In [23]:
utils.pauli_combine(numpy.array([[1, 0, 0, 1],
 [1, 0, 1, 1],
 [1, 1, 1, 0],
 [0, 1, 1, 0]]), numpy.array([[0, 0, 1, 1],
 [1, 1, 0, 1],
 [1, 1, 0, 0],
 [1, 0, 1, 1]]), numpy.array([0, 0, 0, 0]))

0 : [0 3 0 0]
[0 0 0 0] [0 0 1 1]
0
1 : [0 3 0 0]
[0 0 1 1] [1 1 0 1]
1
2 : [0 3 1 0]
[1 1 1 2] [1 1 0 0]
2
3 : [0 3 3 0]
[2 2 1 2] [1 0 1 1]
3


(array([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]), array([0, 3, 3, 0]))

In [26]:
pauli.pauli('IY')@pauli.pauli('YZ')@pauli.pauli('YI')

+iIX

In [27]:
pauli.CliffordMap(r).compose(pauli.CliffordMap(r).inverse())

 +XI
 +ZI
 +IX
 +IZ

In [39]:
utils.acq_mat(r)

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0]])

In [20]:
ps = paulis('XXX','YYY','ZZZ')
ps

 +XXX
 +YYY
 +ZZZ

In [21]:
ps.rotate_by(pauli('IZI'))

 +XYX
 -YXY
 +ZZZ

In [22]:
ps.gs

array([[1, 0, 1, 1, 1, 0],
       [1, 1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0, 1]])

In [23]:
g = pauli('XXIIII').g
numpy.repeat(g.reshape(6,2).sum(axis=-1) > 0,2)

array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False])

In [24]:
p = pauli('Z')
p

 +Z

In [25]:
for p in reversed(ps):
    print(p)

 +ZZZ
 -YXY
 +XYX


In [319]:
g1, g2 = utils.random_pair(50)

In [345]:
%timeit utils.acq(g1, g2)

595 ns ± 35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
pauli('IXX') @ -pauli('IYX')

-iIZI

In [27]:
utils.condense(pauli('IIZII').g)

(array([0, 1]), array([2]))

In [99]:
dict(zip(*numpy.unique(numpy.array([0,0,0,1,2,3]), return_counts=True)))

{0: 3, 1: 1, 2: 1, 3: 1}

In [28]:
gs = utils.random_clifford_tableau(4)
hs = paulis('+IIXI','+YIXX','-IZYZ','+XIIY').gs

In [30]:
stabilizer.ghz_state(4)

StabilizerState(
   +ZZII
   +IZZI
   +IIZZ
   +XXXX)

In [31]:
circ1 = circuit.diagonalize(pauli('IXII'))
circ2 = circuit.diagonalize(pauli('IIYZ'),2)

In [32]:
circ1.compose(circ2)

QuantumCircuit(
  |[0,1]|
  |[0,1][2,3]|)

In [33]:
circ1.compile()
circ1.forward_map

CliffordMap(
  X0-> -ZZII
  Z0-> +IXII
  X1-> +ZIII
  Z1-> +XXII
  X2-> +IIXI
  Z2-> -IIYZ
  X3-> +IIXY
  Z3-> +IIIZ)

In [34]:
os = paulis('IXII','IIYZ')
circ1.forward(os.copy())

 +ZIII
 +IIZI

In [35]:
os.tokenize()

array([[0, 1, 0, 0, 4],
       [0, 0, 2, 3, 4]])